In [54]:
# https://github.com/Coding-Crashkurse/LangGraph-Tutorial/blob/main/highlight_rag.ipynb
# RAG avançado: destacando texto recuperado em PDFs
# https://www.youtube.com/watch?v=GB_LG0QtXGg

# pip install openai
# pip install langchain-openai
# pip install python-dotenv
# pip install pypdf
# pip install langchain-community
# pip install PyMuPDF
# pip install chromadb
# pip install langgraph

from langchain_openai import ChatOpenAI
#from langchain_groq import ChatGroq
#from langchain_anthropic import ChatAnthropic
#from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

In [55]:
load_dotenv(dotenv_path='../.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
#groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
#anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
#gemini_api_key = os.getenv("GEMINI_API_KEY")

In [56]:
from langchain_community.document_loaders import PyPDFLoader # pip install pypdf

loader = PyPDFLoader("output.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [57]:
pages

[Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-04-14T11:22:55-03:00', 'author': 'Antonio Abrantes', 'moddate': '2025-04-14T11:22:55-03:00', 'source': 'output.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='id;Tema;Pergunta;Resposta;Modelos;Despacho \n1;Vício de procedimento;No parecer de indeferimento não houve o tratamento de todas \nas petições apresentadas relacionadas diretamente ao mérito da decisão impugnada, por \nexemplo de existe uma petição subsídio ao exame que foi desconsiderada. Existe um \nvício de porcedimento neste caso?;Sim. O examinador de recurso deve se certificar de \nque todas as petições pertinentes ao exame de mérito foram consideradas no parecer de \nindeferimento. O examinador recursal deve verificar se o quadro reivindicatório analisado \né o correto e se foi desconsiderada alguma das petições apresentadas relacionadas \ndiretamente ao mérito da

In [58]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
    separators=["#"]  # "\n\n",
)

In [59]:
chunks = text_splitter.split_documents(pages)
len(chunks)

74

In [60]:
chunks[1]

Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-04-14T11:22:55-03:00', 'author': 'Antonio Abrantes', 'moddate': '2025-04-14T11:22:55-03:00', 'source': 'output.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='# \n2;Regra de transição;Existe uma regra transitória para adoção dos novos procedimentos \nde recurso de pedidos de patente que possibilite a aceitação de novas vias de quadro \nreivindicatório na fase recursal?;Sim, a Portaria/INPI/Nº 10, de 08 de março de 2024, que \naprova as Diretrizes de instrução de Recursos e Processos Administrativos de Nulidade \nestabelece uma regra de transição. De acordo com esta Portaria/INPI/Nº 10 publicada na \nRPI 2776, para os pedidos de patente nos quais o exame substantivo iniciou após 1º de \nabril de 2024, emendas na segunda instância já não são aceitas. Para os pedidos mais \nantigos é possível se aceitar novas vias de quadro reivi

In [61]:
chunks[1].metadata["source"]

'output.pdf'

In [62]:
chunks[1].page_content

'# \n2;Regra de transição;Existe uma regra transitória para adoção dos novos procedimentos \nde recurso de pedidos de patente que possibilite a aceitação de novas vias de quadro \nreivindicatório na fase recursal?;Sim, a Portaria/INPI/Nº 10, de 08 de março de 2024, que \naprova as Diretrizes de instrução de Recursos e Processos Administrativos de Nulidade \nestabelece uma regra de transição. De acordo com esta Portaria/INPI/Nº 10 publicada na \nRPI 2776, para os pedidos de patente nos quais o exame substantivo iniciou após 1º de \nabril de 2024, emendas na segunda instância já não são aceitas. Para os pedidos mais \nantigos é possível se aceitar novas vias de quadro reivindicatório na fase recursal durante \nesta fase de transição. Segundo o item 7 da mesma Portaria, em conformidade com o que \ndetermina o Despacho Decisório do Presidente do INPI publicado na RPI 2764, de 26 de \ndezembro de 2023, as diretrizes aqui instituídas passam a vigorar a partir de 02 de abril de \n2024. O recu

In [63]:
# pip install PyMuPDF
import fitz  # Agora sim é do PyMuPDF
#doc = fitz.open("output.pdf")
print(fitz.__file__)
#!pip uninstall -y fitz
#!pip install pymupdf

C:\Users\otimi\anaconda3\envs\chromadb\lib\site-packages\fitz\__init__.py


In [64]:
import fitz  # pip install PyMuPDF
from pathlib import Path # crie diretório static
from langchain.schema import Document

def highlight_chunks_in_pdf(documents: list[Document]):
    if not documents:
        print("Nenhum documento entregue. Abortar.")
        return

    pdf_path = documents[0].metadata.get("source")
    if not pdf_path:
        print("Nenhuma 'source' presente nos metadados. Abortar.")
        return

    pdf_file = Path(pdf_path)
    output_path = pdf_file.stem + "_highlighted" + pdf_file.suffix

    doc = fitz.open(pdf_path)

    for document in documents:
        page_number = document.metadata.get("page", 0)
        text_to_highlight = document.page_content

        if page_number < 0 or page_number >= len(doc):
            print(f"A página {page_number} não existe no PDF. Pule esse pedaço.")
            continue

        page = doc[page_number]
        normalized_text = " ".join(text_to_highlight.split())

        matches = page.search_for(normalized_text)

        for match in matches:
            page.add_highlight_annot(match)
    doc.save(output_path)
    doc.close()

    print(f"Os destaques foram salvos em '{output_path}'.")

In [65]:
highlight_chunks_in_pdf(documents=chunks[0:2])

Os destaques foram salvos em 'output_highlighted.pdf'.


In [66]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI # pip install chromadb
from langchain_community.vectorstores import Chroma
from langchain import hub

embedding_function = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = Chroma.from_documents(chunks, embedding_function, persist_directory="nova_pasta_temporaria") 
retriever = db.as_retriever() 


In [67]:
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model="gpt-4o-mini")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm

In [68]:
from typing import TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.schema import Document


class AgentState(TypedDict):
    messages: list[BaseMessage]
    documents: list[Document]

def retrieve(state):
    question = state["messages"][-1].content # Pega a última mensagem enviada pelo usuário (assumindo uma estrutura tipo ChatMessage).
    documents = retriever.invoke(question)
    state["documents"] = documents
    return state

def generate_answer(state):
    question = state["messages"][-1].content
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    print(documents)
    state["messages"].append(generation)
    return state

# O parâmetro state é um dicionário que representa o estado atual do agente. Ele é passado automaticamente de um nó para outro no grafo do LangGraph.
# Esse estado carrega informações compartilhadas entre as etapas do fluxo, como:
# as mensagens trocadas até agora (state["messages"])
# os documentos recuperados (state["documents"])

# state tem uma estrutura assim:
#{
#    "messages": [
#        {"role": "user", "content": "Quem foi Júlio César?"}
#    ],
#    "documents": [...]  # preenchido na etapa 'retrieve'
#}

In [69]:
from langgraph.graph import StateGraph, END # pip install langgraph

workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve)
workflow.add_node("generate_answer", generate_answer)
workflow.add_edge("retrieve", "generate_answer")
workflow.add_edge("generate_answer", END)
workflow.set_entry_point("retrieve")
graph = workflow.compile()

In [70]:
result = graph.invoke(
    input={
        "messages": [HumanMessage(content="O que é causa madura?")]
    }
)
print(result["messages"])

[Document(metadata={'moddate': '2025-04-14T11:22:55-03:00', 'total_pages': 14, 'source': 'output.pdf', 'page_label': '6', 'author': 'Antonio Abrantes', 'creator': 'Microsoft® Word para Microsoft 365', 'page': 5, 'creationdate': '2025-04-14T11:22:55-03:00', 'producer': 'Microsoft® Word para Microsoft 365'}, page_content='# \n21;Parecer recursal;Considere um parecer de indeferimento que não tenha vícios \nformais, mas há vício de julgamento e este provocou prejuízo ao recorrente. O examinador \nde recurso deve necessariamente devolver este pedido à primeira instância para que \nprossiga o exame ou ele mesmo pode deferir este pedido se entender que o pedido \natende aos demais critérios de patenteabilidade? ;O examinador pode dar provimento a \neste recurso e aplicar o princípio da causa madura. Segundo Parecer 0016/2023 \nCGPI/PFE-INPI/PGF/AGU publicado na RPI 2762 de 12/12/2023 item 52. É possível \nvislumbrar a possibilidade de a petição ser imediatamente analisada pela segunda \ninstâ

In [71]:
highlight_chunks_in_pdf(result["documents"])

Os destaques foram salvos em 'output_highlighted.pdf'.


In [72]:
messages = result["messages"]
pergunta = messages[0].content
print("Pergunta:", pergunta)


Pergunta: O que é causa madura?


In [73]:
# Extrair o conteúdo da AIMessage (resposta)
resposta = messages[1].content
print("Resposta:", resposta)

Resposta: Causa madura é um princípio que permite que um examinador deferir um pedido de recurso com base em que este já atende aos critérios de patenteabilidade, sem necessidade de devolução à primeira instância. Se a análise revela um vício de julgamento e a questão está clara e resolvida, o examinador pode decidir diretamente. Esse princípio é aplicado para evitar demoras e garantir decisões mais rápidas e eficazes.
